# Validating and Importing User-Item-Interaction Data <a class="anchor" id="top"></a>

In this notebook, you will choose a dataset and prepare it for use with Amazon Personalize.

1. [How to Use the Notebook](#usenotebook)
1. [Introduction](#intro)
1. [Choose a Dataset or Data Source](#source)
1. [Configure an S3 bucket and an IAM role](#bucket_role)
1. [Create dataset group](#group_dataset)
1. [Create the Interactions Schema](#interact_schema)
1. [Create the Items Schema](#items_schema)
1. [Create the Users Schema](#users_schema)
1. [Import the Interactions Data](#import_interactions)
1. [Import the Items Metadata](#import_items)
1. [Import the User Metadata](#import_users)
1. [Storing Useful Variables](#vars)

## How to Use the Notebook <a class="anchor" id="usenotebook"></a>

The code is broken up into cells like the one below. There's a triangular Run button at the top of this page that you can click to execute each cell and move onto the next, or you can press `Shift` + `Enter` while in the cell to execute it and move onto the next one.

As a cell is executing you'll notice a line to the side showcase an `*` while the cell is running or it will update to a number to indicate the last cell that completed executing after it has finished exectuting all the code within a cell.

Simply follow the instructions below and execute the cells to get started with Amazon Personalize using case optimized recommenders.


## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

In Amazon Personalize, you start by creating a dataset group, which is a container for Amazon Personalize components. Your dataset group can be one of the following:

A Domain dataset group, where you create preconfigured resources for different business domains and use cases, such as getting recommendations for similar videos (VIDEO_ON_DEMAND domain) or best selling items (ECOMMERCE domain). You choose your business domain, import your data, and create Recommenders. You use Recommenders in your application to get recommendations.

Use a [Domain dataset group](https://docs.aws.amazon.com/personalize/latest/dg/domain-dataset-groups.html) if you have a video on demand or e-commerce application and want Amazon Personalize to find the best configurations for your use cases. If you start with a Domain dataset group, you can also add custom resources such as solutions with solution versions trained with recipes for custom use cases.

A [Custom dataset group](https://docs.aws.amazon.com/personalize/latest/dg/custom-dataset-groups.html), where you create configurable resources for custom use cases and batch recommendation workflows. You choose a recipe, train a solution version (model), and deploy the solution version with a campaign. You use a campaign in your application to get recommendations.

Use a Custom dataset group if you don't have a video on demand or e-commerce application or want to configure and manage only custom resources, or want to get recommendations in a batch workflow. If you start with a Custom dataset group, you can't associate it with a domain later. Instead, create a new Domain dataset group.

You can create and manage Domain dataset groups and Custom dataset groups with the AWS console, the AWS Command Line Interface (AWS CLI), or programmatically with the AWS SDKs.

## Define your Use Case <a class="anchor" id="usecase"></a>
[Back to top](#top)

There are a few guidelines for scoping a problem suitable for Personalize. We recommend the values below as a starting point, although the [official limits](https://docs.aws.amazon.com/personalize/latest/dg/limits.html) lie a little lower.

* Authenticated users
* At least 50 unique users
* At least 100 unique items
* At least 2 dozen interactions for each user 

Most of the time this is easily attainable, and if you are low in one category, you can often make up for it by having a larger number in another category.

The user-item-iteraction data is key for getting started with the service. This means we need to look for use cases that generate that kind of data, a few common examples are:

1. Video-on-demand applications
1. E-commerce platforms

Defining your use-case will inform what data and what type of data you need.

In this example we are going to be creating:

1. Amazon Personalize ECOMMERCE Domain recommender for the ["Frequently bought together"](https://docs.aws.amazon.com/personalize/latest/dg/ECOMMERCE-use-cases.html#frequently-bought-together-use-case) use case.
1. Amazon Personalize ECOMMERCE Domain recommender for the ["Recommended for you"](https://docs.aws.amazon.com/personalize/latest/dg/ECOMMERCE-use-cases.html#recommended-for-you-use-case) use case.
1. Amazon Personalize Custom Campaign for a personalized ranked list of items, for instance shelf/rail/carousel based on some information (category, colour, style etc...) 

All of these will be created within the same dataset group and with the same input data.

The diagram bellow shows an overview of what we will be building in this wokshop.

![Workflow](images/image1.png)

In this notebook we will be working on the Data Layer, Creatig a Dataset Group and importing the Datasets. 


## Choose a Dataset or Data Source <a class="anchor" id="source"></a>
[Back to top](#top)

Regardless of the use case, the algorithms all share a base of learning on user-item-interaction data which is defined by 3 core attributes:

1. **UserID** - The user who interacted
1. **ItemID** - The item the user interacted with
1. **Timestamp** - The time at which the interaction occurred

Generally speaking your data will not arrive in a perfect form for Personalize, and will take some modification to be structured correctly. This notebook looks to guide you through all of that. 

Amazon Personalize has certain required fieds and reserved keywords, and you need to format your data to meet these requirements. 

Correctly formated data will allow you to work with filters as well as supporting the [Recommended for you](https://docs.aws.amazon.com/personalize/latest/dg/ECOMMERCE-use-cases.html#recommended-for-you-use-case) Domain Recommender, and complying with the [ECOMMERCE domain dataset and schema requirements](https://docs.aws.amazon.com/personalize/latest/dg/ECOMMERCE-datasets-and-schemas.html#ECOMMERCE-dataset-requirements).

### Open and Explore the Simulated Retail Interactions Dataset

For this example we are using a synthetic datace generated when you deployed your working environment via CloudFormation.

Python ships with a broad collection of libraries and we need to import those as well as the ones installed to help us like [boto3](https://aws.amazon.com/sdk-for-python/) (AWS SDK for python) and [Pandas](https://pandas.pydata.org/)/[Numpy](https://numpy.org/) which are core data science tools.

In [ ]:
# Import Dependencies

import boto3
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
import csv
import yaml

%matplotlib inline

# Setup the Amazon Personalzie client
personalize = boto3.client('personalize')

In [ ]:
# We will upload our training data from these files:
raw_items_filename = "../../automation/ml_ops/domain/Retail/data/Items/items.csv"                # Do Not Change
raw_users_filename = "../../automation/ml_ops/domain/Retail/data/Users/users.csv"                # Do Not Change
raw_interactions_filename = "../../automation/ml_ops/domain/Retail/data/Interactions/interactions.csv"  # Do Not Change
items_filename = "items.csv"                # Do Not Change
users_filename = "users.csv"                # Do Not Change
interactions_filename = "interactions.csv"  # Do Not Change

First let us see a few lines of the raw CSV data:

- An ITEM_ID column of the item interacted with
- A USER_ID column of the user who interacted
- An EVENT_TYPE column which can be used to train different Personalize campaigns and also to filter on recommendations.
- The custom DISCOUNT column which is a contextual metadata field, that Personalize reranking and user recommendation campaigns can take into account to guess on the best next product.
- A TIMESTAMP of when the interaction happened

In [ ]:
interactions_df = pd.read_csv(raw_interactions_filename)
interactions_df.head()

Chart the counts of each `EVENT_TYPE` generated for the interactions dataset. We're simulating a site where visitors heavily view/browse products and to a lesser degree add products to their cart and checkout.

In [ ]:
categorical_attributes = interactions_df.select_dtypes(include = ['object'])

plt.figure(figsize=(16,3))
chart = sns.countplot(data = categorical_attributes, x = 'EVENT_TYPE')
plt.xticks(rotation=90, horizontalalignment='right')
plt.show()

### Explore the Users Dataset

In [ ]:
users_df = pd.read_csv(raw_users_filename)
pd.set_option('display.max_rows', 5)

users_df.head()

We can also leverage some syntectic user metadata to get some additional information about our users. This file was also generated when you deployed your working environment.

In [ ]:
user_metadata_file_name = '../../automation/ml_ops/domain/Retail/metadata/Users/users.json'
zipped_file_name = user_metadata_file_name+'.gz'

# this will delete the zipped file. -k is not supported with the current version of gzip on this instance.
!gzip -d $zipped_file_name

user_metadata_df = pd.read_json (user_metadata_file_name)
user_metadata_df

In [ ]:
# zipping the metadata file
!gzip $user_metadata_file_name

### Explore the Items Dataset

Let us look at the items dataset.

In [ ]:
products_df = pd.read_csv(raw_items_filename)
pd.set_option('display.max_rows', 5)

products_df.head()

We are going to load some additional metadata about each item that will help make the recommended items more readable in this workshop. This is data you'd typically find in an item catalog system/database.

In [ ]:
# load the item meta data
item_metadata_file_name = '../../automation/ml_ops/domain/Retail/metadata/Items/products.yaml'
with open(item_metadata_file_name) as f:
    item_metadata_df = pd.json_normalize(yaml.load(f, Loader=yaml.FullLoader))[['id', 'name', 'category', 'style', 'featured']]
    
display (item_metadata_df)

This function will be used to lookup the name for a product given its ID.

In [ ]:
def get_item_name_from_id ( item_id ):
    item_name = item_metadata_df [item_metadata_df ['id'] == item_id]['name'].values[0]
    return item_name

Let's test the function above by adding an `ITEM_NAME` column to a dataframe that is populated with the name of each product.

In [ ]:
test = products_df.copy()

test['ITEM_NAME'] = test.apply(
      lambda row:
        get_item_name_from_id(row['ITEM_ID'] ), axis=1
  )
display(test.head())

Note the `ITEM_NAME` column has been added to the above data frame. We'll use this function later.

## Configure an S3 bucket and an IAM  role <a class="anchor" id="bucket_role"></a>
[Back to top](#top)

So far, we have downloaded, manipulated, and saved the data onto the Amazon EBS instance attached to instance running this Jupyter notebook.  

By default, the Personalize service does not have permission to acccess the data we uploaded into the S3 bucket in our account. In order to grant access to the  Personalize service to read our CSVs, we need to set a Bucket Policy and create an IAM role that the Amazon Personalize service will assume. Let's set all of that up.

Use the metadata stored on the instance underlying this Amazon SageMaker notebook, to determine the region it is operating in. If you are using a Jupyter notebook outside of Amazon SageMaker, simply define the region as a string below. The Amazon S3 bucket needs to be in the same region as the Amazon Personalize resources we have been creating so far.

In [ ]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print('region:', region)

Amazon S3 bucket names are globally unique. To create a unique bucket name, the code below will append the string `personalizepocvod` to your AWS account number. Then it creates a bucket with this name in the region discovered in the previous cell.

In [ ]:
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-" + region + "-" + "personalize-immersion-day-retail"
print('bucket_name:', bucket_name)
try: 
    if region == "us-east-1":
        s3.create_bucket(Bucket=bucket_name)
    else:
        s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={'LocationConstraint': region}
            )
except:
    print("Bucket already exists. Using bucket", bucket_name)

### Set the S3 bucket policy
Amazon Personalize needs to be able to read the contents of your S3 bucket. So add a bucket policy which allows that.

In [ ]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket",
                "s3:PutObject"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy));

### Create an IAM role

Amazon Personalize needs the ability to assume roles in AWS in order to have the permissions to execute certain tasks. Let's create an IAM role and attach the required policies to it. The code below attaches very permissive policies; please use more restrictive policies for any production application.

In [ ]:
iam = boto3.client("iam")

role_name = account_id+"-PersonalizeS3-Immersion-Day"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

try:
    create_role_response = iam.create_role(
        RoleName = role_name,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
    );
    
except iam.exceptions.EntityAlreadyExistsException as e:
    print('Warning: role already exists:', e)
    create_role_response = iam.get_role(
        RoleName = role_name
    );

role_arn = create_role_response["Role"]["Arn"]
    
print('IAM Role: {}'.format(role_arn))
    
attach_response = iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = "arn:aws:iam::aws:policy/AmazonS3FullAccess"
);

role_arn = create_role_response["Role"]["Arn"]

# Pause to allow role to be fully consistent
time.sleep(30)
print('Done.')

### Upload data to S3

Now that your Amazon S3 bucket has been created, upload the CSV file of our user-item-interaction data. 

In [ ]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(users_filename).upload_file(raw_users_filename)
boto3.Session().resource('s3').Bucket(bucket_name).Object(items_filename).upload_file(raw_items_filename)
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(raw_interactions_filename)

## Create dataset group <a class="anchor" id="group_dataset"></a>
[Back to top](#top)

The highest level of isolation and abstraction with Amazon Personalize is a *dataset group*. Information stored within one of these dataset groups has no impact on any other dataset group or models created from one - they are completely isolated. This allows you to run many experiments and is part of how we keep your models private and fully trained only on your data. 

Before importing the data prepared earlier, there needs to be a dataset group and a dataset added to it that handles the interactions.

Dataset groups can house the following types of information:

* User-item-interactions
* Event streams (real-time interactions)
* User metadata
* Item metadata

We need to create the dataset group that will contain our three datasets.

#### Create Dataset Group
The following cell will create a new dataset group with the name personalize-immersion-day-retail


In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = 'personalize-immersion-day-retail',
    domain='ECOMMERCE'
)
dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

print(f'DatasetGroupArn = {dataset_group_arn}')

#### Wait for Dataset Group to Have ACTIVE Status

Before we can use the dataset group, it must be active. This can take a minute or two. Execute the cell below and wait for it to show the ACTIVE status. It checks the status of the dataset group every 60 seconds, up to a maximum of 3 hours.

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(15)

Now that you have a dataset group, you can create a dataset for the interaction data.

## Create the Interactions Schema <a class="anchor" id="interact_schema"></a>
[Back to top](#top)

Now that we've loaded and prepared our three datasets we'll need to configure the Amazon Personalize service to understand our data so that it can be used to train models for generating recommendations. Amazon Personalize requires a schema for each dataset so it can map the columns in our CSVs to fields for model training. Each schema is declared in JSON using the [Apache Avro](https://avro.apache.org/) format. 

First, define a schema to tell Amazon Personalize what type of dataset you are uploading. There are several reserved and mandatory keywords required in the schema, based on the type of dataset. More detailed information can be found in the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html).

Here, you will create a schema for interactions data, which requires the `USER_ID`, `ITEM_ID`, and `TIMESTAMP` fields. These must be defined in the same order in the schema as they appear in the dataset.

#### Interactions Dataset Schema

The interactions dataset has three required columns: `ITEM_ID`, `USER_ID`, and `TIMESTAMP`. The `TIMESTAMP` represents when the user interated with an item and must be expressed in Unix timestamp format (seconds). For this dataset we also have an `EVENT_TYPE` column that includes multiple common eCommerce event types (`View`, `AddToCart`, `Purchase`, and so on).

The interactions dataset supports metadata columns. Interaction metadata columns are a way to provide contextual details that are specific to an interaction such as the user's current device type (phone, tablet, desktop, set-top box, etc), the user's current location (city, region, metro code, etc), current weather conditions, and so on. For this dataset, we have a `DISCOUNT` column that indicates whether the user is interacting with an item that is currently discounted (`Yes`/`No`). It's being used to learn a user's affinity for items that are on sale or not.

In [ ]:
interactions_schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "EVENT_TYPE",  # "View", "Purchase", etc.
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "DISCOUNT",  # This is the contextual metadata - "Yes" or "No".
            "type": "string",
            "categorical": True,
        },
    ],
    "version": "1.0"
}

try:
    create_schema_response = personalize.create_schema(
        name = "personalize-immersion-day-retail-schema-interactions",
        schema = json.dumps(interactions_schema),
        domain='ECOMMERCE'
    )
    print(json.dumps(create_schema_response, indent=2))
    interactions_schema_arn = create_schema_response['schemaArn']
except personalize.exceptions.ResourceAlreadyExistsException:
    print('You aready created this schema.')
    schemas = personalize.list_schemas(maxResults=100)['schemas']
    for schema_response in schemas:
        if schema_response['name'] == "personalize-immersion-day-retail-schema-interactions":
            interactions_schema_arn = schema_response['schemaArn']
            print(f"Using existing schema: {interactions_schema_arn}")

### Create the interactions dataset

With a schema created, you can create a dataset within the dataset group. Note that this does not load the data yet, but creates a schema of what the data looks like. 

In [ ]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-immersion-day-dataset-interactions",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interactions_schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

## Create the Items schema<a class="anchor" id="items_schema"></a>
[Back to top](#top)

First, define a schema to tell Amazon Personalize what type of dataset you are uploading. There are several reserved and mandatory keywords required in the schema, based on the type of dataset. More detailed information can be found in the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html).

The items dataset schema requires an `ITEM_ID` column and at least one metadata column. Up to 50 metadata columns can be added to the items dataset.

For this dataset we have three metadata columns: `PRICE`, `CATEGORY_L1`, `CATEGORY_L2`, `PRODUCT_DESCRIPTION`, and `GENDER` (see schema definition in the cell below). 

We mapped the `category` and `style` fields from the items catalog to the `CATEGORY_L1` and `CATEGORY_L2` columns to indicate category levels. The `gender_affinity` field used to indicate Women's and Men's products (clothing, footwear, etc) has been mapped to the `GENDER` column in the schema. Note too that `CATEGORY_L1`, `CATEGORY_L2`, and `GENDER` are annotated as being categorical (`"categorical": true`). This tells Personalize to interpret the column value for each row as one or more categorical values where the `|` character can be used to separate values. For example, `value1|value2|value3`. The `PRODUCT_DESCRIPTION` column is annotated as being a text column (`"textual": true`). This tells Personalize that this column contains unstructured text. A natural language processing (NLP) model is used to extract features from the textual column to use as features in the model. Including a textual column in your items dataset can significantly enhance the relevancy of recommendations. Currently only one textual column can be included in the items dataset and the text must be in English. For more information, please refer to [the documentation](https://docs.aws.amazon.com/personalize/latest/dg/items-datasets.html#text-data).

In [ ]:
items_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "PRICE",
            "type": "float"
        },
        {
            "name": "CATEGORY_L1",
            "type": "string",
            "categorical": True,
        },
        {
            "name": "CATEGORY_L2",
            "type": "string",
            "categorical": True,
        },
        {
            "name": "PRODUCT_DESCRIPTION",
            "type": "string",
            "textual": True
        },
        {
            "name": "GENDER",
            "type": "string",
            "categorical": True,
        },
    ],
    "version": "1.0"
}

try:
    create_schema_response = personalize.create_schema(
        name = "personalize-immersion-day-retail-schema-items",
        schema = json.dumps(items_schema),
        domain='ECOMMERCE'
    )
    items_schema_arn = create_schema_response['schemaArn']
    print(json.dumps(create_schema_response, indent=2))
except personalize.exceptions.ResourceAlreadyExistsException:
    print('You aready created this schema.')
    schemas = personalize.list_schemas(maxResults=100)['schemas']
    for schema_response in schemas:
        if schema_response['name'] == "personalize-immersion-day-retail-schema-items":
            items_schema_arn = schema_response['schemaArn']
            print(f"Using existing schema: {items_schema_arn}")

### Create Items Dataset
With a schema created, you can create a dataset within the dataset group. Note that this does not load the data yet, but creates a schema of what the data looks like. 

In [ ]:
dataset_type = "ITEMS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-immersion-day-dataset-items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = items_schema_arn
)

items_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

## Create the Users schema<a class="anchor" id="users_schema"></a>
[Back to top](#top)

First, define a schema to tell Amazon Personalize what type of dataset you are uploading. There are several reserved and mandatory keywords required in the schema, based on the type of dataset. More detailed information can be found in the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html).

Here, you will create a schema for user data, which requires the `USER_ID`, and an additonal metadata field. For this dataset we have metadata columns for `AGE` and `GENDER`. These must be defined in the same order in the schema as they appear in the dataset.

In [ ]:
users_schema = {
    "type": "record",
    "name": "Users",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "AGE",
            "type": "int"
        },
        {
            "name": "GENDER",
            "type": "string",
            "categorical": True,
        }
    ],
    "version": "1.0"
}

try:
    create_schema_response = personalize.create_schema(
        name = "personalize-immersion-day-retail-schema-users",
        schema = json.dumps(users_schema),
        domain='ECOMMERCE'
    )
    print(json.dumps(create_schema_response, indent=2))
    users_schema_arn = create_schema_response['schemaArn']
except personalize.exceptions.ResourceAlreadyExistsException:
    print('You aready created this schema, seemingly')
    schemas = personalize.list_schemas(maxResults=100)['schemas']
    for schema_response in schemas:
        if schema_response['name'] == "personalize-immersion-day-retail-schema-users":
            users_schema_arn = schema_response['schemaArn']
            print(f"Using existing schema: {users_schema_arn}")

### Create Users Dataset

In [ ]:
dataset_type = "USERS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-immersion-day-dataset-users",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = users_schema_arn
)

users_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

Let's wait untill all the datasets have been created.

In [ ]:
%%time

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_response = personalize.describe_dataset(
        datasetArn = interactions_dataset_arn
    )
    status =  describe_dataset_response["dataset"]['status']
    print("Interactions Dataset: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
while time.time() < max_time:
    describe_dataset_response = personalize.describe_dataset(
        datasetArn = items_dataset_arn
    )
    status =  describe_dataset_response["dataset"]['status']
    print("Items Dataset: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
while time.time() < max_time:
    describe_dataset_response = personalize.describe_dataset(
        datasetArn = users_dataset_arn
    )
    status =  describe_dataset_response["dataset"]['status']
    print("Users Dataset: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## Import the Interactions Data <a class="anchor" id="import_interactions"></a>
[Back to top](#top)

Earlier you created the dataset group and dataset to house your information, so now you will execute an import job that will load the interactions data from the S3 bucket into the Amazon Personalize dataset. 

In [ ]:
interactions_create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "immersion-day-retail-dataset-interactions-import",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

interactions_dataset_import_job_arn = interactions_create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(interactions_create_dataset_import_job_response, indent=2))

## Import the Item Metadata <a class="anchor" id="import_items"></a>
[Back to top](#top)

Earlier you created the dataset group and dataset to house your information, now you will execute an import job that will load the item data from the S3 bucket into the Amazon Personalize dataset. 

In [ ]:
items_create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "immersion-day-retail-dataset-items-import",
    datasetArn = items_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, items_filename)
    },
    roleArn = role_arn
)

items_dataset_import_job_arn = items_create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(items_create_dataset_import_job_response, indent=2))

## Import the User Metadata <a class="anchor" id="import_users"></a>
[Back to top](#top)

Earlier you created the dataset group and dataset to house your information, now you will execute an import job that will load the user data from the S3 bucket into the Amazon Personalize dataset. 

In [ ]:
users_create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "immersion-day-retail-dataset-users-import",
    datasetArn = users_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, users_filename)
    },
    roleArn = role_arn
)

users_dataset_import_job_arn = users_create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(users_create_dataset_import_job_response, indent=2))

### Wait for Import Jobs to Complete

Before we can use the dataset, the import job must be active. Execute the cell below and wait for it to show the ACTIVE status. It checks the status of the import job every minute, up to a maximum of 6 hours.

It will take 10-15 minutes for the import jobs to complete. While you're waiting you can learn more about Datasets and Schemas in [the documentation](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html).

We will wait for all three jobs to finish.

In [ ]:
%%time

import_job_arns = [ items_dataset_import_job_arn, users_dataset_import_job_arn, interactions_dataset_import_job_arn ]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for job_arn in reversed(import_job_arns):
        import_job_response = personalize.describe_dataset_import_job(
            datasetImportJobArn = job_arn
        )
        status = import_job_response["datasetImportJob"]['status']

        if status == "ACTIVE":
            print(f'Import job {job_arn} successfully completed')
            import_job_arns.remove(job_arn)
        elif status == "CREATE FAILED":
            print(f'Import job {job_arn} failed')
            if import_job_response.get('failureReason'):
                print('   Reason: ' + import_job_response['failureReason'])
            import_job_arns.remove(job_arn)

    if len(import_job_arns) > 0:
        print('At least one dataset import job still in progress')
        time.sleep(60)
    else:
        print("All import jobs have ended")
        break

With all imports now complete you can  start training recommenders and solutions. Run the cell below before moving on to store a few values for usage in the next notebooks. After completing that cell open notebook `02_Training_Layer.ipynb` to continue.

## Storing Useful Variables <a class="anchor" id="vars"></a>
[Back to top](#top)

Before exiting this notebook, run the following cells to save the version ARNs for use in the next notebook.

In [ ]:
%store items_dataset_arn
%store interactions_dataset_arn
%store users_dataset_arn
%store dataset_group_arn
%store bucket_name
%store role_arn
%store role_name
%store region
%store item_metadata_df
%store user_metadata_df
%store products_df